In [1]:
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd

cluster = LocalCluster()
client = Client(cluster)

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54020,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.88 GiB
Comm: tcp://127.0.0.1:54054,Total threads: 2
Dashboard: http://127.0.0.1:54057/status,Memory: 3.97 GiB
Nanny: tcp://127.0.0.1:54025,


In [3]:
generated_data = 'generated_data/'
parquet_file_path = generated_data +  "speaker_attributes.parquet"

In [4]:
%time parquet_df = dd.read_parquet(parquet_file_path)

Wall time: 43 ms


In [5]:
parquet_df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 15 entries, aliases to religion
dtypes: object(14), int64(1)

In [6]:
%time parquet_df.head()

Wall time: 2.67 s


,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


In [7]:
# %time mem_usage = parquet_df.memory_usage(deep=True).sum()
# %time mem_usage.compute()

In [8]:
parquet_df.npartitions

16

In [9]:
# parquet_df=parquet_df.repartition(npartitions=50)

In [10]:
# parquet_df.npartitions

In [11]:
%time quotebank_2019_sample = dd.read_json('generated_data/quotes-2019-nytimes.json.bz2', lines=True, compression='bz2')
# %time quotebank_2019_sample.head()

Wall time: 15.3 s


In [12]:
quotebank_2019_sample.divisions

(None, None)

In [13]:
quotebank_2019_sample = quotebank_2019_sample.repartition(npartitions=1)
# making sure this is only one partition

In [14]:
quotebank_2019_sample.divisions

(None, None)

In [15]:
# import pandas as pd 

In [16]:
def first_qid(x):
    if x:
        x = x[0]
    else:
        x = '0'
    return x

def clean_qids(df):
    return df.apply(first_qid, meta=('qids', 'str'))
%time quotebank_2019_sample_clean =  quotebank_2019_sample.copy()
%time quotebank_2019_sample_clean['qids'] = clean_qids(quotebank_2019_sample['qids'])

Wall time: 2 ms
Wall time: 9.99 ms


In [17]:
#%time quotebank_2019_sample_clean = quotebank_2019_sample_clean.compute()

In [18]:
%time quotebank_2019_sample_clean = quotebank_2019_sample_clean.set_index('qids')

Wall time: 17 s


In [19]:
# %time quotebank_2019_sample_clean.head()

In [20]:
quotebank_2019_sample_clean.divisions

('0', 'Q99860')

In [21]:
%time parquet_df = parquet_df.set_index('id')

Wall time: 15.3 s


In [22]:
parquet_df.divisions

('Q10000001',
 'Q102302700',
 'Q12787847',
 'Q168705',
 'Q20047184',
 'Q26883815',
 'Q42319672',
 'Q45546466',
 'Q5667633',
 'Q58657858',
 'Q65853031',
 'Q75388272',
 'Q76258554',
 'Q89055259',
 'Q94514816',
 'Q96083341',
 'Q9999999')

In [24]:
quotebank_2019_sample_clean.divisions

('0', 'Q99860')

In [25]:
%time merged_df = quotebank_2019_sample_clean.merge(parquet_df, left_index=True, right_index=True)

Wall time: 17 ms


In [26]:
%time merged_df.compute()

Wall time: 3min 57s


,quoteID,quotation,speaker,date,numOccurrences,probas,urls,phase,aliases,date_of_birth,...,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,label,candidacy,type,religion
Q1000491,2019-03-27-072428,"No way will something like that happen again,",Bud Collins,2019-03-27 18:00:13,1,"[[Bud Collins, 0.7254], [None, 0.2341], [Mark ...",[http://nytimes.com/2019/03/27/sports/baseball...,E,"[Arthur Collins, Arthur Worth Collins, Jr.]",[+1929-06-17T00:00:00Z],...,1392361030,None,None,"[Q13219447, Q10833314, Q13219424, Q36180, Q159...",None,None,Bud Collins,None,item,None
Q100071,2019-02-20-046794,"If we only ever end up with Breck, he's amazing,",Kikkan Randall,2019-02-20 16:01:44,4,"[[Kikkan Randall, 0.7914], [None, 0.1974], [Je...",[https://www.nytimes.com/2019/02/20/sports/kik...,E,None,[+1982-12-31T00:00:00Z],...,1378098700,None,None,[Q13382608],None,None,Kikkan Randall,None,item,None
Q100071,2019-02-20-129442,"When cancer happened, my mom decided that bott...",Kikkan Randall,2019-02-20 16:01:44,4,"[[Kikkan Randall, 0.6162], [None, 0.3596], [Je...",[https://www.nytimes.com/2019/02/20/sports/kik...,E,None,[+1982-12-31T00:00:00Z],...,1378098700,None,None,[Q13382608],None,None,Kikkan Randall,None,item,None
Q100071,2019-02-20-029579,"He's hanging out in Seattle in storage, and th...",Kikkan Randall,2019-02-20 16:01:44,4,"[[Kikkan Randall, 0.6887], [Jeff Ellis, 0.2124...",[https://www.nytimes.com/2019/02/20/sports/kik...,E,None,[+1982-12-31T00:00:00Z],...,1378098700,None,None,[Q13382608],None,None,Kikkan Randall,None,item,None
Q100071,2019-02-20-129637,"When I found out I had cancer, it was almost d...",Kikkan Randall,2019-02-20 16:01:44,4,"[[Kikkan Randall, 0.7406], [None, 0.2342], [Je...",[https://www.nytimes.com/2019/02/20/sports/kik...,E,None,[+1982-12-31T00:00:00Z],...,1378098700,None,None,[Q13382608],None,None,Kikkan Randall,None,item,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q99860,2019-11-15-084418,"The climate package is, at best, an indication...",Ottmar Edenhofer,2019-11-15 17:50:29,3,"[[Ottmar Edenhofer, 0.7694], [None, 0.2306]]",[http://www.nytimes.com/2019/11/15/world/europ...,E,None,[+1961-01-01T00:00:00Z],...,1387080028,None,None,"[Q188094, Q1622272]",None,"[Q849697, Q5978719]",Ottmar Edenhofer,None,item,None
Q99860,2019-11-15-013008,but this has not yet taken place.,Ottmar Edenhofer,2019-11-15 17:50:29,3,"[[Ottmar Edenhofer, 0.7766], [None, 0.2234]]",[http://www.nytimes.com/2019/11/15/world/europ...,E,None,[+1961-01-01T00:00:00Z],...,1387080028,None,None,"[Q188094, Q1622272]",None,"[Q849697, Q5978719]",Ottmar Edenhofer,None,item,None
Q99860,2019-09-20-081341,The German government has failed to protect th...,Ottmar Edenhofer,2019-09-20 17:32:31,1,"[[Ottmar Edenhofer, 0.6558], [None, 0.2552], [...",[http://www.nytimes.com/2019/09/20/world/europ...,E,None,[+1961-01-01T00:00:00Z],...,1387080028,None,None,"[Q188094, Q1622272]",None,"[Q849697, Q5978719]",Ottmar Edenhofer,None,item,None
Q99860,2019-09-20-087470,"The whole package lacks courage,",Ottmar Edenhofer,2019-09-20 17:32:31,1,"[[Ottmar Edenhofer, 0.7113], [None, 0.2293], [...",[http://www.nytimes.com/2019/09/20/world/europ...,E,None,[+1961-01-01T00:00:00Z],...,1387080028,None,None,"[Q188094, Q1622272]",None,"[Q849697, Q5978719]",Ottmar Edenhofer,None,item,None


In [27]:
%time merged_df.head()

Wall time: 41.2 s


,quoteID,quotation,speaker,date,numOccurrences,probas,urls,phase,aliases,date_of_birth,...,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,label,candidacy,type,religion
